In this notebook we will be pushing a trained model to the Hugging Face Hub.

In [ ]:
%%capture
!pip install transformers
!pip install huggingface_hub

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Let's get the model from the local directory where it is saved

model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/custom_BERT_model")

In [ ]:
# Now let's push the model to the Hugging Face hub

model.push_to_hub("arjbaid/Fine_Tuned_BERT_Classifier")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/arjbaid/Fine_Tuned_BERT_Classifier/commit/29066f6ec138dbe82383cf3e37719cb1a3397a56', commit_message='Upload BertForSequenceClassification', commit_description='', oid='29066f6ec138dbe82383cf3e37719cb1a3397a56', pr_url=None, pr_revision=None, pr_num=None)

Now let's load the model from our Hugging Face hub repo and test the model on sample comments.

In [ ]:
# Loading the model from our repo

bert_model = BertForSequenceClassification.from_pretrained("arjbaid/Fine_Tuned_BERT_Classifier")

In [ ]:
sample = ["Are you crazy?", 
          "Wow! What a fast car!", 
          "screw you!!", 
          "I loved you but not anymore since I am now over you, so kindly leave me alone.",
          "The outage was first reported on the afternoon of May 30, and since then, users have been reporting outages."]

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer(sample, padding=True, truncation=True, return_tensors="pt").to(device)

output = bert_model(**inputs)

# Printing the model output logits
print(output.logits)

tensor([[-2.3967,  2.6050],
        [ 1.5576, -1.6723],
        [-3.7019,  3.6106],
        [ 1.7837, -2.1087],
        [ 3.6249, -3.2847]], grad_fn=<AddmmBackward0>)


In [ ]:
# Let's convert the model output logits into probabilites by passing them through the Softmax function

predictions = torch.softmax(output.logits, dim=-1)
print(predictions)
preds_array = predictions.detach().cpu().numpy()

# Displaying the model predictions

for i, pred in enumerate(preds_array):
  if pred[0] > pred[1]:
    print("\nComment:", sample[i],"\nPrediction: Not Toxic", "\nProbability:", pred[0])
  else:
    print("\nComment:", sample[i],"\nPrediction: Toxic", "\nProbability:", pred[1])

tensor([[6.6816e-03, 9.9332e-01],
        [9.6194e-01, 3.8056e-02],
        [6.6671e-04, 9.9933e-01],
        [9.8001e-01, 1.9989e-02],
        [9.9900e-01, 9.9712e-04]], grad_fn=<SoftmaxBackward0>)

Comment: Are you crazy? 
Prediction: Toxic 
Probability: 0.99331844

Comment: Wow! What a fast car! 
Prediction: Not Toxic 
Probability: 0.96194446

Comment: screw you!! 
Prediction: Toxic 
Probability: 0.9993333

Comment: I loved you but not anymore since I am now over you, so kindly leave me alone. 
Prediction: Not Toxic 
Probability: 0.980011

Comment: The outage was first reported on the afternoon of May 30, and since then, users have been reporting outages. 
Prediction: Not Toxic 
Probability: 0.9990029
